# How to use the GQLAlchemy query builder? 

Through this short tutorial, you will learn how to install Memgraph, connect to it from a Jupyter Notebook and use the GQLAlchemy library to build Cypher queries.

## 1. Prerequisites

For this tutorial, you will need to install:
- [Jupyter](https://jupyter.org/install)
- [Docker](https://docs.docker.com/get-docker/)

Docker is used because Memgraph is a native Linux application and cannot be installed on Windows and macOS.

## 2. Installation using Docker

After you install Docker, you can set up Memgraph by running:

```
docker run -it -p 7687:7687 -p 3000:3000 memgraph/memgraph-platform
```

This command will start the download and after it finishes, run the Memgraph container.

## 3. Connecting to Memgraph with GQLAlchemy

We will be using the **GQLAlchemy** object graph mapper (OGM) to connect to Memgraph and execute **Cypher** queries easily. GQLAlchemy also serves as a Python driver/client for Memgraph. You can install it using:

```
pip install gqlalchemy
```

> **Hint**: You may need to install [CMake](https://cmake.org/download/) before installing GQLAlchemy.

Maybe you got confused when I mentioned Cypher. You can think of Cypher as SQL for graph databases. It contains many of the same language constructs like `CREATE`, `UPDATE`, `DELETE`... and it's used to query the database.

In [26]:
from gqlalchemy import Memgraph

In [27]:
memgraph = Memgraph("127.0.0.1", 7687)

Let's make sure that Memgraph is empty before we start with anything else.

In [28]:
memgraph.drop_database()

## 4. Define a graph schema

We are going to create Python classes that will represent our graph schema. This way, all the objects that are returned from Memgraph will be of the correct type if the class definition can be found.

In [29]:
from typing import Optional
from gqlalchemy import Field, Node, Relationship


class User(Node):
    id: int = Field(index=True, unique=True, db=memgraph)
    name: Optional[str] = Field()
        
class Follows(Relationship, type='FOLLOWS'):
    pass

## 5. Creating and returning nodes

We are going to create a `User`, save it to the database and return it to our program:

In [30]:
from gqlalchemy import create


results = create().node(labels="User", id=0, name="Ron", variable="user").return_().execute()
user = next(results)
print(user)

{'user': <User id=1979 labels={'User'} properties={'id': 0, 'name': 'Ron'}>}


What’s going on here:

- The class `create()` is the base for the query. Other classes can also be imported and used as bases: `match()`, `merge()`, `call()`, `unwind()` and `with()`.
- The method `node()` creates a node with the label `User` and properties `id` and `name`.
- You need to specify the `variable` argument in order to return the results.

## 6. Finding nodes

First, we will need to create a class that will inherit from `gqlalchemy.Relationship`. Then, we can just create instances of the class and call their `save()` method to store them in the database:

In [32]:
from gqlalchemy import match


results = match().node(id=0, variable="user").return_().execute()
user = next(results)
print(user)

{'user': <User id=1979 labels={'User'} properties={'id': 0, 'name': 'Ron'}>}


What’s going on here:

- The class `create()` is the base for the query. It's used to find nodes and relationships in the database.


## What's next?

Now it's time for you to use Memgraph on a graph problem!

You can always check out [Memgraph Playground](https://playground.memgraph.com/) for some cool use cases and examples.
If you need some help with GQLAlchemy, don't forget to check out the docs